Importando Bibliotecas

In [33]:
import numpy as np
import random as rd

Inicializando a população com 1000 indivíduos, sendo cada um deles uma lista de 8 números aleatórios de 0 a 7, que representam a posição das rainhas em cada coluna.

In [34]:
num_individuos = 1000
num_rainhas = 8
populacao_inicial = np.zeros((num_individuos, num_rainhas), dtype=int)

for i in range(num_individuos):
  for j in range(num_rainhas):
    populacao_inicial[i][j] = rd.randrange(8)

Definindo a função de fitness, que compara todos os pares de rainhas e somam na variável f caso elas não se ataquem

In [35]:
def fitness(ind):
  
  f = 0 #fitness

  for i in range(num_rainhas-1):
    for j in range(i+1, num_rainhas):
      # Caso não estejam na mesma linha ou na mesma diagonal
      if ind[i] != ind[j] and abs(i-j) != abs(ind[i]-ind[j]):
        f += 1

  return f

def retorna_fitness(populacao):
  
  fit = []
  
  for i in range(num_individuos):
    f = fitness(populacao[i])
    fit.append(f)

  return fit # Retorna uma lista de fitness dos indivíduos da população
  

Definindo função de seleção por torneio. Nela, são escolhidos três indivíduos aleatórios da população e o com maior fitness é guardado na variável "ind1". Depois, o processo se repete e um outro indivíduo é armazenado na variável "ind2".

In [36]:
def selecao_torneio(fitnesses):
  
  ind1 = -1
  ind2 = -1

  while ind1 == ind2:
    # Torneio 1
    sorteados = rd.sample(range(0, num_individuos), 3)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]] and fitnesses[sorteados[0]] > fitnesses[sorteados[2]]:
      ind1 = sorteados[0]
    elif fitnesses[sorteados[1]] > fitnesses[sorteados[2]]:
      ind1 = sorteados[1]
    else:
      ind1 = sorteados[2]

    # Torneio 2
    sorteados = rd.sample(range(0, num_individuos), 3)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]] and fitnesses[sorteados[0]] > fitnesses[sorteados[2]]:
      ind2 = sorteados[0]
    elif fitnesses[sorteados[1]] > fitnesses[sorteados[2]]:
      ind2 = sorteados[1]
    else:
      ind2 = sorteados[2]
    

  return ind1,ind2

Definindo função de cruzamento de 1 ponto. Nela, é escolhido um ponto aleatório para a realização do cruzamento dos indivíduos escolhidos na parte da seleção, gerando dois filhos.

In [37]:
def cruzamento(ids, populacao):
  
  ponto = rd.randrange(1, num_rainhas)

  pai1 = populacao[ids[0]]
  pai2 = populacao[ids[1]]

  filho1 = np.concatenate([pai1[:ponto], pai2[ponto:]])
  filho2 = np.concatenate([pai2[:ponto], pai1[ponto:]])

  return filho1,filho2


Definindo a função de elitismo, que retorna uma lista dos 10 indivíduos com maior fitness na população, para que eles passem direto para a próxima geração.

In [38]:
def elitismo(fitnesses):

  id = []

  for i in range(10):
    id.append(fitnesses.index(max(fitnesses)))
    fitnesses.pop(id[i])

  return id

Definindo a função de mutação. Nela, o elemento de uma posição aleatória é trocado por um número aleatório. Isso não acontece todas as vezes, dependendo da taxa de mutação definida, que é a chance que um indivíduo tem de sofrer mutação.

In [39]:
def mutacao(filhos, taxa):

  for i in range(len(filhos)):
    if rd.random() < taxa:
      pos = rd.randrange(num_rainhas)
      filhos[i][pos] = rd.randrange(8)

  return filhos


Código principal

In [40]:
# Número de gerações
num_geracoes = 100
fitness_otimo = 28
repeticoes = 0
fit_anterior = 0

for it in range(num_geracoes):

  # Nova populacao vazia
  nova_populacao = np.zeros((num_individuos, num_rainhas), dtype=int)

  # Calcula o fitness da população
  fit = retorna_fitness(populacao_inicial)

  # Imprime o melhor individuo
  id_melhor = fit.index(max(fit))
  print(f"{it+1}a Geração: [", '  '.join(map(str,populacao_inicial[id_melhor])), "] ; Fit =", fit[id_melhor])
  print()

  # Contagem de repetições do maior fitness
  if(fit_anterior == fit[id_melhor]):
    repeticoes += 1
  else:
    repeticoes = 0

  fit_anterior = fit[id_melhor]

  # Condições de parada
  if fit[id_melhor] == fitness_otimo or repeticoes == 20:
    break

  # Elitismo
  elite = elitismo(fit.copy()) # Copiando por causa da referencia do vetor

  for i in range(10):
    nova_populacao[i] = populacao_inicial[elite[i]]

  # Gera os filhos restantes para completar a população
  num_filhos = 2
  while num_filhos < num_individuos:

    # Seleção por torneio
    ind_vencedores = selecao_torneio(fit)
    
    # Cruzamento
    filhos = cruzamento(ind_vencedores, populacao_inicial)

    # Mutacao (50% de chance de mutação em cada indivíduo)
    filhos = mutacao(filhos, 0.1)

    # Coloca os filhos na nova população
    nova_populacao[num_filhos] = filhos[0]
    nova_populacao[num_filhos+1] = filhos[1]

    # Aumenta o número de filhos
    num_filhos = num_filhos + 2

  # Substitui a populacao antiga pela atual
  populacao_inicial = nova_populacao.copy()


1a Geração: [ 3  5  3  1  6  7  2  4 ] ; Fit = 26

2a Geração: [ 3  5  3  1  6  7  2  4 ] ; Fit = 26

3a Geração: [ 3  5  3  1  6  7  2  4 ] ; Fit = 26

4a Geração: [ 3  5  3  1  6  7  2  4 ] ; Fit = 26

5a Geração: [ 7  2  4  6  0  5  3  1 ] ; Fit = 27

6a Geração: [ 7  2  4  6  0  5  3  1 ] ; Fit = 27

7a Geração: [ 7  2  4  6  0  5  3  1 ] ; Fit = 27

8a Geração: [ 7  2  4  6  0  5  3  1 ] ; Fit = 27

9a Geração: [ 7  2  4  6  0  5  3  1 ] ; Fit = 27

10a Geração: [ 7  2  4  6  0  5  3  1 ] ; Fit = 27

11a Geração: [ 7  2  4  6  0  5  3  1 ] ; Fit = 27

12a Geração: [ 4  6  0  3  1  7  5  2 ] ; Fit = 28

